In [1]:
%%bash
pip install transformers -q

In [2]:
import os
import cv2
import torch
import pickle
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from transformers import AutoFeatureExtractor, SwinModel, SwinConfig

In [3]:
def load_image(base_path: str, filename: str) -> np.ndarray:
    return cv2.cvtColor(src=cv2.imread(os.path.join(base_path, filename), cv2.IMREAD_COLOR), code=cv2.COLOR_BGR2RGB)

le = LabelEncoder()

NAMES = [
    "swin-tiny-patch4-window7-224",
    "swin-small-patch4-window7-224",
    "swin-base-patch4-window7-224",
    "swin-base-patch4-window12-384",
]

In [4]:
train_df = pd.read_csv("../input/sorghum-id-fgvc-9/train_cultivar_mapping.csv")

broken_images = pickle.load(open("../input/fgvc9-broken-images/broken_image_names.pkl", "rb"))
for broken_image in broken_images:
    index = train_df.index[train_df.image == broken_image]
    train_df = train_df.drop(index=index)
    
filenames = train_df.image.copy().values
labels    = train_df.cultivar.copy().values 

labels = le.fit_transform(labels)
np.save("labels.npy", labels)

features = np.zeros((len(filenames), 768))

In [5]:
pretrained = AutoFeatureExtractor.from_pretrained("microsoft/" + NAMES[0])
model = SwinModel.from_pretrained("microsoft/" + NAMES[0])



for filename in filenames:
    image = load_image("../input/sorghum-id-fgvc-9/train_images", filename)
    inputs = pretrained(image, return_tensors="pt")
    with torch.no_grad():
        output = model(inputs["pixel_values"])
    features = np.concatenate((features, output.last_hidden_state[:, 0, :]), axis=0)

np.save("features.npy", features)

Downloading:   0%|          | 0.00/70.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/108M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/swin-tiny-patch4-window7-224 were not used when initializing SwinModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing SwinModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SwinModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
